# Growth rates - World

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from matplotlib.dates import DateFormatter

## Data

### COVID-19 data

Source: [Our World in Data](https://ourworldindata.org/coronavirus)

Download updated data:

In [22]:
#url = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
#filename = 'data/world_' + url.split("/")[-1]

#with open(filename, "wb") as f:
#    r = requests.get(url)
#    f.write(r.content)

Or load a local file until a specific date:

In [23]:
filename = 'data/world_owid-covid-data_06_08.csv'

In [24]:
df = pd.read_csv(filename)
df.head()

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand
0,AFG,Asia,Afghanistan,2019-12-31,0,0,0,0,0.0,0.0,...,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5
1,AFG,Asia,Afghanistan,2020-01-01,0,0,0,0,0.0,0.0,...,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5
2,AFG,Asia,Afghanistan,2020-01-02,0,0,0,0,0.0,0.0,...,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5
3,AFG,Asia,Afghanistan,2020-01-03,0,0,0,0,0.0,0.0,...,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5
4,AFG,Asia,Afghanistan,2020-01-04,0,0,0,0,0.0,0.0,...,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5


Growh rate raw calculation:

In [25]:
def delta(df_conf):

    list_ = []
    list_.append(0)

    for j in range(len(df_conf) - 1):

        list_.append(df_conf[j+1] - df_conf[j])

    return list_

Select a list of countries:

In [29]:
# All
countries_ = df['location'].unique()

# Specify
#countries_ = np.array(['Brazil', 'Germany'])

In [30]:
results_ = []

### EWM
alpha = 30.
alpha2 = 7.
alpha3 = 5.

### Classification threshold
# Cases
CASES_threshold = 50
# Normalized growth rate (%)
GROWTH_RATE_threshold = 1
# Normalized growth acceleration (%)
GROWTH_ACCEL_threshold = 0.01

### Last date into results_ list?
last = True
### If not, what date? (Do no forget to set last = False)
#### CAREFUL: a location might not have data for this particular day (some places do not have records on sundays, for instance)
#until_date = '2020-05-02'

for coutries in countries_:

    print(coutries)

    df_ = df[ df['location'] == coutries].sort_values(by='date').set_index('date')
    df_ =  df_ [ df_['total_cases']  > 0]
    df_.index = pd.to_datetime(df_.index)

    # Rate
    df_['growth_rate_'] = delta(df_['total_cases'])

    # Exponential weight
    df_['growth_rate'] = df_['growth_rate_'].ewm(com= alpha).mean()

    # Rate
    df_['growth_accel_'] = delta(df_['growth_rate'])

    # Exponential weight
    df_['growth_accel'] = df_['growth_accel_'].ewm(com= alpha2).mean()

    # Rate
    df_['growth_accel_rate_'] = delta(df_['growth_accel'])

    # Exponential weight
    df_['growth_accel_rate'] = df_['growth_accel_rate_'].ewm(com= alpha3).mean()

    # Dropping unsmoothed quantities
    df_ = df_.drop('growth_rate_', axis=1)
    df_ = df_.drop('growth_accel_', axis=1)
    df_ = df_.drop('growth_accel_rate_', axis=1)

    # Normalized quantities
    df_['growth_rate_NORM_(%)'] = 100*df_['growth_rate'] / df_['total_cases']
    df_['growth_accel_NORM_(%)'] = 100*df_['growth_accel'] / df_['total_cases']
    df_['growth_accel_rate_NORM_(%)'] = 100*df_['growth_accel_rate'] / df_['total_cases']

    # Classification

    df_['LENTO'] =  (df_['total_cases'] < CASES_threshold)*1

    df_['EXPONENCIAL'] = ( df_['growth_accel_NORM_(%)']  >= GROWTH_ACCEL_threshold )*1

    df_['DESACELERACAO'] = ( df_['growth_accel_NORM_(%)']  <= -GROWTH_ACCEL_threshold )*1

    df_['ESTAVEL'] = ( (df_['growth_rate_NORM_(%)']  <  GROWTH_RATE_threshold) &  (df_['growth_accel_NORM_(%)']  > - GROWTH_ACCEL_threshold) & (df_['growth_accel_NORM_(%)']  <         GROWTH_ACCEL_threshold) )*1

    df_['LINEAR'] = ( (df_['growth_rate_NORM_(%)']  >=  GROWTH_RATE_threshold ) &  (df_['growth_accel_NORM_(%)']  > - GROWTH_ACCEL_threshold) & (df_['growth_accel_NORM_(%)']  < GROWTH_ACCEL_threshold) )*1


    df_.loc[df_['LENTO'] == True, 'CLASSIFICACAO'] = 'LENTO'
    df_.loc[ (df_['LENTO'] != True) & (df_['DESACELERACAO'] == True), 'CLASSIFICACAO'] = 'DESACELERACAO'
    df_.loc[ (df_['LENTO'] != True) & (df_['ESTAVEL'] == True), 'CLASSIFICACAO'] = 'ESTAVEL'
    df_.loc[ (df_['LENTO'] != True) & (df_['LINEAR'] == True), 'CLASSIFICACAO' ] = 'LINEAR'
    df_.loc[ (df_['LENTO'] != True) & (df_['EXPONENCIAL'] == True), 'CLASSIFICACAO' ] = 'EXPONENCIAL'


    # Plot
    #fig, axes = plt.subplots(1, 4, figsize= (24, 4))
    
        
    #axes[0].plot(df_['growth_rate'])
    #axes[0].set_title(coutries, fontsize= 16)
    #axes[0].set_ylabel('Daily New Cases', fontsize= 13)
    #axes[0].set_xlabel('Date', fontsize= 13)
    #axes[0].grid(linestyle=':')
    #axes[0].legend()

    #axes[1].plot(df_['growth_accel'], color= 'C3')
    #axes[1].set_title(coutries, fontsize= 16)
    #axes[1].set_ylabel('Growth acceleration', fontsize= 13)
    #axes[1].set_xlabel('Date', fontsize= 13)
    #axes[1].grid(linestyle=':')
    #axes[1].legend()

    #axes[2].plot(df_['growth_accel_rate'], color= 'C2')
    #axes[2].set_title(coutries, fontsize= 16)
    #axes[2].set_ylabel('Growth acceleration rate', fontsize= 13)
    #axes[2].set_xlabel('Date', fontsize= 13)
    #axes[2].grid(linestyle=':')
    #axes[2].legend()

    #axes[3].plot(df_[['total_cases']], 'o-', color= 'C1', label= 'Official data')
    #axes[3].set_ylabel('Confirmed cases', fontsize= 12)
    #axes[3].set_title(coutries, fontsize= 16)
    #axes[3].set_xlabel('Date', fontsize= 13)
    #axes[3].grid(linestyle=':')
    #axes[3].legend()


    #date_form = DateFormatter("%m/%d")
    #axes[0].xaxis.set_major_formatter(date_form)
    #axes[1].xaxis.set_major_formatter(date_form)
    #axes[2].xaxis.set_major_formatter(date_form)
    #axes[3].xaxis.set_major_formatter(date_form)

    df_ = df_.reset_index()


    if last:
        results_.append(df_.iloc[-1].to_dict())
    else:
        idx_ = df_.index[df_['date'] == until_date]
        if len(idx_) > 0:
            results_.append(df_.iloc[idx_[0]].to_dict())
        else:
            print('%s data NOT available for %s' % (until_date, countries))

    #plt.savefig('file_name.png', bbox_inches='tight')
    #plt.show()


results = pd.DataFrame(results_)

#results.to_csv('results/dfs/growth_world_class_.csv', index= True, sep= ';')

Afghanistan
Albania
Algeria
Andorra
Angola
Anguilla
Antigua and Barbuda
Argentina
Armenia
Aruba
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bermuda
Bhutan
Bolivia
Bonaire Sint Eustatius and Saba
Bosnia and Herzegovina
Botswana
Brazil
British Virgin Islands
Brunei
Bulgaria
Burkina Faso
Burundi
Cambodia
Cameroon
Canada
Cape Verde
Cayman Islands
Central African Republic
Chad
Chile
China
Colombia
Comoros
Congo
Costa Rica
Cote d'Ivoire
Croatia
Cuba
Curacao
Cyprus
Czech Republic
Democratic Republic of Congo
Denmark
Djibouti
Dominica
Dominican Republic
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Ethiopia
Faeroe Islands
Falkland Islands
Fiji
Finland
France
French Polynesia
Gabon
Gambia
Georgia
Germany
Ghana
Gibraltar
Greece
Greenland
Grenada
Guam
Guatemala
Guernsey
Guinea
Guinea-Bissau
Guyana
Haiti
Honduras
Hong Kong
Hungary
Iceland
India
Indonesia
Iran
Iraq
Ireland
Isle of Man
Israel
Italy
Jamaica
Japan
Jersey
Jordan
Kazakhstan